In [1]:
import foursquare as fs
import pandas as pd
import time

In [2]:
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_id = 'site_of_your_app'

client = fs.Foursquare(client_id=client_id, client_secret=client_secret, 
                       redirect_uri=redirect_id)

In [3]:
# returns a list of venues in a given area
def venues_list(client, ne, sw):
    
    # see also https://developer.foursquare.com/categorytree
    categories = '4d4b7105d754a06374d81259,4d4b7105d754a06376d81259'
    
    vens = client.venues.search({'intent' : 'browse', 'limit' : 1000,
                                 'ne' : ne, 'sw' : sw,
                                 'categoryId' : categories})
    
    return [venue['id'] for venue in vens['venues']]

In [4]:
# returns a pandas DF that contains information about
# Discrict (that you choose), BarId, UserId, Review text,
# whether a visitor made a photo, and visitor's reaction
def reviews(client, VENUE_ID_LIST, district_name):
    
    df = pd.DataFrame(columns=['District', 'BarId', 'UserId', 'Review', 
                               'MadePhoto', 'Reaction'])
    
    for VENUE_ID in VENUE_ID_LIST:
        reviews = client.venues.tips(VENUE_ID, params={'limit':1000, 
                                                       'sort':'popular'})
        
        for review in reviews['tips']['items']:
            UserId = review['user']['id']
            MadePhoto = False if review.get('photo', 0) == 0 else True
            Review = review['text']
            Reaction = review.get('authorInteractionType', None)
            
            df = df.append([{'District' : district_name, 
                             'BarId' : VENUE_ID, 'UserId' : UserId, 
                             'Review' : Review, 'MadePhoto' : MadePhoto,
                             'Reaction' : Reaction}])
        
    return df

In [67]:
# returns a pandas DF with selected user's personal information
# given list of user's IDs
# free API limit is 500 (every query resets in an hour).
def user_info(client, user_id_list):
    
    df = pd.DataFrame(columns=['UserId', 'HomeCity', 'Gender', 'FriendCount',
                               'CheckInCount', 'ListCount', 'PhotoCount', 
                               'TipCount'])
    # count is for deleted users (removed their accont but left a review)
    count = 0
    # count_user is for printing the progress (see try-clause)
    count_user = 0
    for user in user_id_list:
        
        try:
            person = client.users(user)['user']
            
            count_user += 1
            print(count_user, end=',')
            
            HomeCity = person['homeCity']
            Gender = person['gender']
            Friends = person['friends']['count']
            Chekins = person['checkins']['count']
            Lists = person['lists']['count']
            Photos = person['photos']['count']
            Tips = person['tips']['count']
            
            df = df.append([{'UserId' : user, 'HomeCity' : HomeCity, 
                             'Gender' : Gender, 'FriendCount' : Friends,
                             'CheckInCount' : Chekins, 'ListCount' : Lists, 
                             'PhotoCount' : Photos, 'TipCount': Tips}])
        
        # runs if the acc of a user has been deleted
        except fs.ParamError:
            count += 1
            print('ParamError. Deleted', user)
            continue
        
        # runs if the API limit prevents new queries (sleep one hour)
        except fs.RateLimitExceeded:
            print('waiting for an hour...')
            time.sleep(60 * 60)
            print('continue')
            
            person = client.users(user)['user']
            
            HomeCity = person['homeCity']
            Gender = person['gender']
            Friends = person['friends']['count']
            Chekins = person['checkins']['count']
            Lists = person['lists']['count']
            Photos = person['photos']['count']
            Tips = person['tips']['count']
            
            df = df.append([{'UserId' : user, 'HomeCity' : HomeCity, 
                             'Gender' : Gender, 'FriendCount' : Friends,
                             'CheckInCount' : Chekins, 'ListCount' : Lists, 
                             'PhotoCount' : Photos, 'TipCount': Tips}])
        
    print('Deleted:', count, 'of', len(user_id_list))
    return df

In [6]:
# specifying coordinates of districts
# Note: only 50 venues will be reported within chosen area 
# in functions I choose some atbitrary large constant --- 'limit' : 1000
dum_coords = {'sw' : '59.929506,30.320345', 'ne' : '59.935408,30.334722'}
zhu_coords = {'ne' : '59.939031,30.361372', 'sw' : '59.933129,30.346996'}
rub_coords = {'sw' : '59.926377,30.336009', 'ne' : '59.932366,30.350343'}

dum_venues = venues_list(client, ne=dum_coords['ne'], sw=dum_coords['sw'])
zhu_venues = venues_list(client, ne=zhu_coords['ne'], sw=zhu_coords['sw'])
rub_venues = venues_list(client, ne=rub_coords['ne'], sw=rub_coords['sw'])

In [8]:
# downloading reviews for districts
dum_data = reviews(client, VENUE_ID_LIST=dum_venues, district_name='Dumskaya')
zhu_data = reviews(client, VENUE_ID_LIST=zhu_venues, district_name='Zukovskogo')
rub_data = reviews(client, VENUE_ID_LIST=rub_venues, district_name='Rubinshteyna')

In [74]:
# concatenating all districts in one DF
# data = pd.read_csv('venues_meta.csv')
data = pd.concat([dum_data, zhu_data, rub_data])

In [56]:
# selecting only unique users
visitors_lst = list(set(data.UserId))

In [76]:
# downloading information for given users
# users_meta = pd.read_csv('users_meta.csv')
users_meta = user_info(client, user_id_list=visitors_lst)

In [77]:
users_meta.head()

,UserId,HomeCity,Gender,FriendCount,CheckInCount,ListCount,MajorCount,PhotoCount,TipCount
0,57748591,Saint-Petersburg,female,84.0,1833.0,4.0,0.0,0.0,26.0
1,77174927,"Saint Petersburg, Russia",female,19.0,640.0,19.0,0.0,0.0,4.0
2,105804316,"Moscow, Moscow",female,0.0,4.0,2.0,0.0,0.0,5.0
3,13726205,Санкт-Петербург,female,265.0,5082.0,4.0,0.0,0.0,22.0
4,21611581,City of St. Petersburg,male,175.0,4043.0,2.0,0.0,0.0,11.0


In [78]:
# merging the user's meta and venues tips data
data_users = pd.merge(left=data, right=users_meta, how='outer', on='UserId')

In [79]:
data_users.head()

,District,BarId,UserId,Review,MadePhoto,Reaction,HomeCity,Gender,FriendCount,CheckInCount,ListCount,MajorCount,PhotoCount,TipCount
0,Dumskaya,52b89801498eb5da45f72173,142661708,"Как в городе с хорошими локальными, интересным...",False,disliked,Saint Petersburg,female,0.0,0.0,3.0,0.0,0.0,113.0
1,Dumskaya,5257e2c011d238157c415649,142661708,"Подозреваю, что стригут тут все же лучше, чем ...",False,meh,Saint Petersburg,female,0.0,0.0,3.0,0.0,0.0,113.0
2,Zukovskogo,57691bb8cd10287c73f4dc3b,142661708,Неплохая и более здоровая вариация на тему нац...,False,meh,Saint Petersburg,female,0.0,0.0,3.0,0.0,0.0,113.0
3,Zukovskogo,538493ea498e5da6509017a4,142661708,Непревзойдённый флэт уайт),False,liked,Saint Petersburg,female,0.0,0.0,3.0,0.0,0.0,113.0
4,Zukovskogo,521772ba11d24a4864be2b6e,142661708,"Хорошее место, несмотря на то, что сетевое. Вр...",False,liked,Saint Petersburg,female,0.0,0.0,3.0,0.0,0.0,113.0
